In [7]:
# 네이버 영화평 감성분석

# 한글이외의 문자를 모두 공백으로 처리하고 strip 꼭꼬고꼬고
train_df.document = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣㄱ-힣]', ' ' ).str.strip()
train_df.head(3)

In [8]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [15]:
train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep='\t')
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", sep='\t')
train_df.head(3)
# label 이 긍정 부정 . 0 부정 1긍정 

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


1. ### 데이터 전처리

- 트레인 데이터셋 
 - 중복여부 확인

In [16]:
# Null 데이터가 있는지 확인  # 1건 있어서 제거 
train_df.document.isna().sum()

5

In [20]:
# Null 데이터 제거
# train_df.document.dropna(how='any', inplace=True)  # how 가 애니면 정글러인가?
train_df.dropna(how='any', inplace=True)  
train_df.shape

(146182, 3)

In [21]:
# 중복여부 확인
train_df.document.nunique()

146182

In [22]:
# 듀플리케이트 중복 제거 
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146182, 3)

In [23]:
# 긍정 부정 데이터 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

In [25]:
# 이제 테스트 데이터 # 널3개 #
test_df.document.isna().sum()


3

In [27]:
test_df.dropna(how='any', inplace=True)
test_df.shape

(49997, 3)

In [28]:
# 중복확인  # 약 800 건중복 . 4999 - 49157 하면 대략
test_df.document.nunique()

49157

In [29]:
# 중복 제거 
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49157, 3)

In [30]:
# label 데이터 분포 # 긍정 부정. 
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

### 2. 텍스트 전처리

여기까지 하면 참 좋겠는데 또 해야할게 있어 바로~~ 텍스트 전처리 

- train 데이터셋 

In [31]:
# 한글이외의 문자를 모두 공백으로 처리하고 strip
train_df.document = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣㄱ-힣]', ' ' ).str.strip()
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


- 영어로만 감상평 쓴 사람들 다 날아가 
- 또한 구두점 없으면 문장단위 분석 어려운데, 단어로 쪼개니 결국 상관없나?

In [32]:
# Null 값은 아니라는데 , np.nan 이용
review = 'What a good movie!!!'
review1='                     '
review1.strip()

''

# 수치 다르다 제길

In [40]:
# ''만 남은 데이터 --> np.nan 으로 변경시킨 후 제거 
train_df.document.replace('', np.nan, inplace=True) ## 스트링 리플레이스 아님 주의 
train_df.document.isna().sum()

0

In [41]:
# 널 데이터 제거
train_df.dropna(how='any', inplace=True)
train_df.shape

# 수업은 145393

(145409, 3)

In [43]:
test_df.document.replace('', np.nan, inplace=True) ## 스트링 리플레이스 아님 주의 
test_df.document.isna().sum()
test_df.dropna(how='any', inplace=True)
test_df.shape
# 수치는 다르다. 

(48857, 3)

- 전처리가 끝난 데이터는 저장
 - index = False로 주는것 주의 

In [47]:
train_df.to_csv('data/naver_movie_train_전처리완료.csv', sep='\t', index=False)
test_df.to_csv('data/naver_movie_test_전처리완료.csv', sep='\t', index=False)

In [ ]:
CV 카운트 벡터라이즈로도 해보고 
tf-idf 로도 해볼것. 어떤 데이터 꺼내야 하니 ? 
다운로드 보다 전처리후 다운이 편할 것 같아서 . 

In [ ]:
전처리 중복값 날리고, 변환하고(영어 삭제 등 ) 그후 결측치 다시 보고 저장. 

### 3. 한글 처리  뭘로 SOYNLP

아까 만들어둔 스코어 사용할 것 .

최대 점수 토크나이저는 뜨이ㅓ쓰기 되지 않는 문장에서 점수가 높은 글자 시퀀스를 순차적으로 찾아내는 토크나이저\

아아.. 띄어쓰기 제대로 안된좀.. 

In [49]:
import joblib
scores = joblib.load('data/scores.pkl')

In [51]:
from soynlp.tokenizer import MaxScoreTokenizer

max_tokenizer = MaxScoreTokenizer(scores)

In [52]:
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정'
max_tokenizer.tokenize(text)

['교도소', '이야기', '구먼', '솔직', '히', '재미는', '없다', '평점', '조정']

영어단어 벡터화 
한글 - > 형태소 -> 벡터화 

In [ ]:
# 우리 아직 학습 안시켰다 주의 